In [37]:
import backtrader as bt
import ccxt
from LAB import key
import pandas as pd

binance = ccxt.binance(config = {
    'apiKey': key.api_key, 
    'secret': key.secret_key,
    'enableRateLimit' : True,
    'options': {
        'defaultType': 'future'
    }
})



class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    # 출력해주는 함수
    def log(self, txt, doprint=False):
        
        #파라미터만 간단히 바꿀 땐 로그 안띄움.
        if  doprint:
            dt = self.datas[0].datetime.date(0)
            dt_time = self.datas[0].datetime.time()
            
            #시스템에도 로그 띄움
            print('%s, %s, %s' % (dt.isoformat(),dt_time, txt))
            
            return
            #파일에 로그 씀
            # 파일 열기
            f = open('./log/strategy.txt', 'a',encoding='UTF-8')

            # 파일에 텍스트 쓰기
            f.write('%s, %s' % (dt.isoformat(), txt))
            f.write('\n')
            
            # 파일 닫기
            f.close()

    #초기화 함수
    def __init__(self):
        #항상 종가로 계산 !
        self.dataclose = self.datas[0].close

        # 주문 현황과 거래된 가격, 그리고 수수료 기록
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # 전략 우위 판별을 위해 승률 기록
        self.trade_count =0
        self.trade_win_count = 0

        # 이평선 전략을 위해 그려봄
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    # 이것도 이벤트 함수로 next 가기 이전에 들르게 됨.
    def notify_order(self, order):
        
        #거래 대기 중이면 그냥 가만히 있음
        if order.status in [order.Submitted, order.Accepted]:
            return

        
        # 거래 됨 -> 기록해야함.
        if order.status in [order.Completed]:
            if order.isbuy():
                # 이떄 사실 시간도 알려줘야함
                self.log(
                    '샀음, Price: %.2f, 총 거래량: %.2f, 거래 수수료 : %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm), True)

                # 수수료도 따로 저장해둠 어딘가에 쓰겠지뭐
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            
            #팔았다고 반대로 기록
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            #이건 지금은 필요 없는데 몇개 뒤에 팔건지를 위해 해놓은거
            self.bar_executed = len(self)

        # 거래 취소되거나 돈 없었으면
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('거래 취소됨',True)

        # 거래 되거나 취소 된 경우 대기 중 거래 없으므로 none 넣어줌
        self.order = None

    # 이벤트 함수 개념으로 거래 되면 발생함.
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(('trade 됨 , 거래 수익 %.2f, 거래 순수익) %.2f', 
                 trade.pnl, trade.pnlcomm),True)
        
        # 결국 수익이 났는지 확인, 용어가 정확하지 않아서 헷갈림 
        if trade.pnlcomm > 0:
            self.trade_win_count+=1

    def next(self):
        # 다음 신호로 넘어왔으면 이제 계속 기록
        self.log('현재종가, %.2f' % self.dataclose[0])
    
        # 거래 대기 중인지 확인
        if self.order:
            return

        # 거래 돼서 포지션에 들어가있는지 확인
        if not self.position:

            # 사는 조건
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
                self.trade_count +=1

        else:
            # 파는 조건
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
                
    # 이건 여러 개 전략 검토할 때 쓰는 함수로서 발동 조건은 : 내가볼 떄 함수 끝까지 왔을 떄인듯
    def stop(self):
        print('지금 함수 발동 됨!')
        Ending_value=self.broker.getvalue()
        self.log('(MA Period %2d) 순수익 %.2f' %
                 (self.params.maperiod, Ending_value-Start_Value), doprint=True)
        print('승률', round(self.trade_win_count/self.trade_count*100),'%', '이긴 횟수 : ', self.trade_win_count, '전체 거래 횟수 : ', self.trade_count)
        

# Create a cerebro entity
cerebro = bt.Cerebro()
# Add a strategy
#strats = cerebro.optstrategy(
#    TestStrategy,
#    maperiod=range(10, 31))
#
cerebro.addstrategy(TestStrategy)

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
btc_ohlcv = binance.fetch_ohlcv("BTC/USDT", '15m')
df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
df['datetime'] =  pd.to_datetime(df['datetime'], unit='ms')
df.set_index('datetime', inplace=True)
data = bt.feeds.PandasData(dataname=df)


cerebro.adddata(data)

Start_Value = 10000000

cerebro.broker.setcash(Start_Value)

cerebro.addsizer(bt.sizers.FixedSize, stake=10)

cerebro.broker.setcommission(commission=0)

cerebro.run(maxcpus=1)

2022-12-13, 06:30:00, 샀음, Price: 17185.20, Cost: 171852.00, Comm 0.00
2022-12-13, 06:45:00, ('trade 됨 , 거래 수익 %.2f, NET(거래 순수익 or 거래 수수료) %.2f', '살 때 낸 수수료 %.2f', -384.00000000001455, -384.00000000001455, 0.0)
2022-12-13, 08:30:00, 샀음, Price: 17172.50, Cost: 171725.00, Comm 0.00
2022-12-13, 09:00:00, ('trade 됨 , 거래 수익 %.2f, NET(거래 순수익 or 거래 수수료) %.2f', '살 때 낸 수수료 %.2f', -230.0, -230.0, 0.0)
2022-12-13, 09:15:00, 샀음, Price: 17172.10, Cost: 171721.00, Comm 0.00
2022-12-13, 17:00:00, ('trade 됨 , 거래 수익 %.2f, NET(거래 순수익 or 거래 수수료) %.2f', '살 때 낸 수수료 %.2f', 5818.000000000029, 5818.000000000029, 0.0)
2022-12-13, 19:45:00, 샀음, Price: 17760.60, Cost: 177606.00, Comm 0.00
2022-12-13, 20:15:00, ('trade 됨 , 거래 수익 %.2f, NET(거래 순수익 or 거래 수수료) %.2f', '살 때 낸 수수료 %.2f', -430.99999999998545, -430.99999999998545, 0.0)
2022-12-13, 20:30:00, 샀음, Price: 17750.00, Cost: 177500.00, Comm 0.00
2022-12-13, 22:30:00, ('trade 됨 , 거래 수익 %.2f, NET(거래 순수익 or 거래 수수료) %.2f', '살 때 낸 수수료 %.2f', -100.99999999998545, -100.9